## Core Tests

Test the functionality of the basic audio/MIDI handling features of the app.

- Loading audio/MIDI
- Playback audio/MIDI

In [ ]:
%load_ext autoreload
%autoreload 2

In [5]:
import sys
sys.path.append('..') # leave the notebooks folder into root as the main

from app.core.audio.AudioData import AudioData
from app.core.audio.AudioPlayer import AudioPlayer
from app.core.midi.MidiData import MidiData
from app.core.midi.MidiPlayer import MidiPlayer
from app.core.midi.MidiSynth import MidiSynth

from app.algorithms.pitch.PYin import PYin
from app.algorithms.align.OnsetDf import UserOnsetDf, MidiOnsetDf
from app.config import AppConfig
from app.core.recording.PitchDf import PitchDf, PitchConfig
from app.algorithms.align.DTW import DTW
from app.algorithms.align.CQT import CQT

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



## CORE: MidiData, MidiPlayer, MidiSynth
Load in a MIDI file and create a `MidiData` instance to turn it into a dataframe of messages to be parsed in other parts of the program.

You can play the MIDI to inspect that it sounds right by first loading a `MidiSynth` instance (+ providing a soundfont file), loading your MidiData into a `MidiPlayer` instance, and then running `play()`.

In [6]:
# To inspect how MIDI file is currently parsed
MIDI_FILEPATH = 'data/midi_files/fugue_midi.mid'
midi_data = MidiData(MIDI_FILEPATH)

# Initialize a MidiSynth + MidiPlayer with a given soundfont
SOUNDFONT = 'data/MuseScore_General.sf3'
midi_synth = MidiSynth(soundfont_path=SOUNDFONT)
midi_player = MidiPlayer(midi_synth)

# Play the midi file
midi_player.load_midi(midi_data)
midi_player.play(start_time=0)

Loading MidiSynth...
Synth + soundfont loaded.


## CORE: AudioData, AudioPlayer

Load in an audio file (.mp3) and create a `AudioData` instance to convert it into a numpy array of samples which can be played out using an `AudioPlayer` instance which uses the `sounddevice` package to play the audio in a separate thread.

Uses the sample rate configured in `app/config.py`, which is set to 44100 as default.

In [7]:
AUDIO_FILEPATH = 'data/audio_files/user_fugue2.mp3'
user_audio = AudioData(audio_filepath=AUDIO_FILEPATH, sample_rate=44100)

audio_player = AudioPlayer()
audio_player.load_audio_data(user_audio) # load the player
audio_player.play()